# Parametros de Despacho de Generacion

## Referencias

[1] ....

[2] ....

## Definiciones

* CG: Central de Generacion, puede tener una o varias UGen
* UGen: Unidad de Generacion, corresponde al conjunto turbina, generador y transformador de potencia

## Captura de Datos

Para iniciar con el procesamiento de datos, se toma la referencia de los datos del Centro de Control asi:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# Lectura de archivo con informacion de Fallas
# Para referencia se deja el de despacho de generacion

df_CG = pd.read_excel("~/Repos/CurvasTermicas.xlsx", sheet_name='CG_24h')
df_CG.head()

,fecha,numDiaMes,hora_dia,despachoMW,CeGen,mes,reftemp
0,2024-01-01,1,0,38.300000,TEO,ene,2024-01-01 00:00:00
1,2024-01-01,1,1,38.300000,TEO,ene,2024-01-01 01:00:00
2,2024-01-01,1,2,38.680952,TEO,ene,2024-01-01 02:00:00
3,2024-01-01,1,3,38.800000,TEO,ene,2024-01-01 03:00:00
4,2024-01-01,1,4,38.800000,TEO,ene,2024-01-01 04:00:00


In [11]:
# Graficar tendencia por CG en grafico independiente

# creacion de Fig usando Plotly GO subplots para tenre varias tendencias cada una en su propio subplot
# Creacion de una figura con 4 subplots 
fig = make_subplots(rows= 2, cols= 2)

# Agregas las series al gráfico
fig.add_trace(go.Scatter(x=df_CG['reftemp'].loc[df_CG['CeGen']=='TEO'], 
                         y=df_CG['despachoMW'].loc[df_CG['CeGen']=='TEO'], mode='lines', name='UGen0'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_CG['reftemp'].loc[df_CG['CeGen']=='TES1'], 
                         y=df_CG['despachoMW'].loc[df_CG['CeGen']=='TES1'], mode='lines', name='UGen1'), row=1, col=2)

fig.add_trace(go.Scatter(x=df_CG['reftemp'].loc[df_CG['CeGen']=='TES2'], 
                         y=df_CG['despachoMW'].loc[df_CG['CeGen']=='TES2'], mode='lines', name='UGen2'), row=2, col=1)

fig.add_trace(go.Scatter(x=df_CG['reftemp'].loc[df_CG['CeGen']=='TES3'], 
                         y=df_CG['despachoMW'].loc[df_CG['CeGen']=='TES3'], mode='lines', name='UGen3'), row=2, col=2)

fig.update_layout(title_text= 'Despacho Centros de Generacion')
#fig.update_traces(['UGen1', 'UGen2', 'UGen3', 'UGen4'])
fig.show()

## Estadisticas Fallas

A continuacion se realizaran los calculos basicos de no disponibilidad total y energia total generada por cada unidad

$
NoDisp_{CG}=\frac{\Sigma h_{off}}{\Sigma h_{total}}
$

$
En_{sum\_CG}=\Sigma En_{sum\_hora}
$

In [5]:
NoDisp_TEO=100*df_CG['despachoMW'].loc[(df_CG['CeGen']=='TEO')&(df_CG['despachoMW']<=0.5)].count()/df_CG['despachoMW'].loc[(df_CG['CeGen']=='TEO')].count()
NoDisp_TES1=100*df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES1')&(df_CG['despachoMW']<=0.5)].count()/df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES1')].count()
NoDisp_TES2=100*df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES2')&(df_CG['despachoMW']<=0.5)].count()/df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES2')].count()
NoDisp_TES3=100*df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES3')&(df_CG['despachoMW']<=0.5)].count()/df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES3')].count()

ES_TEO = df_CG['despachoMW'].loc[(df_CG['CeGen']=='TEO')].sum()
ES_TES1 = df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES1')].sum()
ES_TES2 = df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES2')].sum()
ES_TES3 = df_CG['despachoMW'].loc[(df_CG['CeGen']=='TES3')].sum()
ES_CG = ES_TEO + ES_TES1 + ES_TES2 + ES_TES3

delta_t = df_CG['despachoMW'].loc[(df_CG['CeGen']=='TEO')].count()

print(f'En un periodo de {delta_t}_h del 2024 se generaron {ES_CG}_MWh asi:')
print(f'Para TEO, la energia suministrada fue {ES_TEO}_MWh ({100*ES_TEO/ES_CG}_% del despacho) con una No Disp {NoDisp_TEO}_%')
print(f'Para TES1, la energia suministrada fue {ES_TES1}_MWh ({100*ES_TES1/ES_CG}_% del despacho) con una No Disp {NoDisp_TES1}_%')
print(f'Para TES2, la energia suministrada fue {ES_TES2}_MWh ({100*ES_TES2/ES_CG}_% del despacho) con una No Disp {NoDisp_TES2}_%')
print(f'Para TES3, la energia suministrada fue {ES_TES3}_MWh ({100*ES_TES3/ES_CG}_% del despacho) con una Disp {NoDisp_TES3}_%')


En un periodo de 5112_h del 2024 se generaron 535486.163725203_MWh asi:
Para TEO, la energia suministrada fue 196477.088094681_MWh (36.69134730351459_% del despacho) con una No Disp 9.076682316118935_%
Para TES1, la energia suministrada fue 117007.44055226399_MWh (21.850693533943303_% del despacho) con una No Disp 18.91627543035994_%
Para TES2, la energia suministrada fue 116724.59640347_MWh (21.797873467253563_% del despacho) con una No Disp 13.165101721439749_%
Para TES3, la energia suministrada fue 105277.038674788_MWh (19.660085695288537_% del despacho) con una Disp 2.0735524256651017_%


Una vez identificadas los anteriores datos basicos del despacho de generacion, se procede a graficar la curva de disponibilidad acumulada de despacho por UGen.

In [18]:
# Probabilidad acuulada de despacho por CG
fig = px.ecdf(df_CG, x="despachoMW", color="CeGen", width=820, height=420, labels={'TEO': 'UGen0'})
fig.update_layout(title_text= 'Distribucion Acumulada de Despacho por UGen')
fig.update_layout(yaxis_title="Probabilidad")
fig.update_layout(xaxis_title="Despacho Diario [MWh]")

# Probabilidad acumulada del despacho de los CG
# fig = px.ecdf(df_CG, x="despachoMW")
fig.show()

Una vez visualizados los datos por UGen, se consolidan por los CG

In [7]:
# AGrupamiento de datos de Ugen en CGs
grouped_df = df_CG[['fecha', 'despachoMW']].groupby(['fecha']).sum()
grouped_df.head()

,despachoMW
fecha,
2024-01-01,2251.271217
2024-01-02,2251.114300
2024-01-03,1619.883752
2024-01-04,1339.311082
2024-01-05,1323.348411


In [8]:
#Presentacion estadisticos basicos
grouped_df.describe()

,despachoMW
count,213.000000
mean,2514.019548
std,416.572262
min,1305.582043
25%,2207.821357
50%,2738.147441
75%,2788.050293
max,3106.472745


In [9]:

fig = px.ecdf(grouped_df , x="despachoMW", width=980, height=500)
fig.update_layout(title_text= 'Distribucion Acumulada de Despacho de los CG')
fig.update_layout(yaxis_title="Probabilidad")
fig.update_layout(xaxis_title="Despacho Diario [MWh]")

fig.show()

Tomando como referencia que el $~68\%$ de los datos de una distribucion normal se encuentran en el rango $[\mu-\sigma, \mu+\sigma]$, es decir una desviacion estandar arriba y abajo de la media de la muestra, se realizo dicho analisis tomando como referencia los datos entregados por `grouped_df.describe()`, teniendose que $~81.7\%$ de los datos se encuentran en el rango $[2098.01, 2930.01] MWh$.

Ahora tomando como referencia el valor maximo posible de energia generada de $3576 \frac{MWh}{dia}$, para las 5112 horas evaluadas durante 2024 se puede concluir que:

* El $~81.7\%$ del tiempo ($4176.5 h$) el despacho de generacion estuvo en un rango de 833.14MWh ($2\sigma$ ) entre $[2098.01, 2930.01] \frac{MWh}{dia}$.
* El $~92.01\%$ de las 5112 horas evaluadas, es decir, durante 4703.5h no supero el 81.9% de la capacidad de generacion instalada.
* Solo el $~7.9\%$ del tiempo, es decir, $408.44 h$ o 17 dias se tuvo una generacion superior a los $2930.01 \frac{MWh}{dia}$ equivalentes a $122.1 MW$
* El maximo despacho de potencia hasta el 31-jul-24 fue de $129.4 MW$ equivalente al $86.8\%$ de la capacidad instalada utilizable
